In [1]:
from config import POSTGRES_CONNECTION_STRING
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
from IPython.display import clear_output

In [2]:
objects = pd.read_csv('../data/objects-with-coordinates.csv')
objects.head()

,id Объекта,Объект,id Ведомственной Организации,Ведомственная Организация,Доступность,Доступность.1,Широта (Latitude),Долгота (Longitude)
0,100000,Дворец культуры «Капотня»,237332.0,Департамент культуры города Москвы,3.0,Районное,55.638114,37.799177
1,100002,Спортивный комплекс «Полярная звезда»,219165.0,Москомспорт.,3.0,Районное,55.883669,37.653730
2,100004,"Парк, озелененная городская территория «Парк к...",237332.0,Департамент культуры города Москвы,2.0,Окружное,55.753895,37.551559
3,100005,Досуговый центр «Старт-7»,237424.0,Управа Пресненского района города Москвы,3.0,Районное,55.761741,37.544064
4,100012,Дворовая территория,237339.0,Префектура Центрального административного окру...,4.0,Шаговая доступность,55.733356,37.671283


In [3]:
to_update = objects[['id Объекта', 'Широта (Latitude)', 'Долгота (Longitude)']]
to_update.columns = ['id', 'lat', 'lng']
to_update.head()

,id,lat,lng
0,100000,55.638114,37.799177
1,100002,55.883669,37.653730
2,100004,55.753895,37.551559
3,100005,55.761741,37.544064
4,100012,55.733356,37.671283


In [4]:
def con():
    return create_engine(POSTGRES_CONNECTION_STRING)

In [5]:
pers = pd.read_sql_query('''
    select id
    from objects
''', con())
pers.head()

,id
0,120007
1,120008
2,120009
3,120010
4,120011


In [6]:
to_update.shape, pers.shape

((13169, 3), (13169, 1))

In [7]:
def update_object_coordinates(df):
    cn = con()
    batch_size = 100
    sqls = []
    for i, (_, row) in enumerate(df.iterrows()):
        id = row['id']
        lat = row['lat']
        lng = row['lng']
        if np.isnan(lat):
            print(f'ID = {id}')
            continue
        sql = f'''
            update objects
            set lat = {lat},
                lng = {lng}
            where id = {id}
        '''
        sqls.append(sql)
        if len(sqls) >= batch_size:
            print(i)
            sum_sql = ';'.join(sqls)
            cn.execute(sum_sql)
            sqls = []
    sum_sql = ';'.join(sqls)
    cn.execute(sum_sql)

In [8]:
update_object_coordinates(to_update)

99
199
299
399
499
599
699
799
899
999
1099
1199
1299
1399
1499
1599
1699
1799
1899
1999
2099
2199
2299
2399
2499
2599
2699
2799
2899
2999
3099
3199
3299
3399
3499
3599
3699
3799
3899
3999
4099
4199
4299
4399
4499
4599
4699
4799
4899
4999
5099
5199
5299
5399
5499
5599
5699
5799
5899
5999
6099
6199
6299
6399
6499
6599
6699
6799
6899
6999
7099
7199
7299
7399
7499
7599
7699
7799
7899
7999
8099
8199
8299
8399
8499
8599
8699
8799
8899
8999
9099
9199
9299
9399
9499
9599
9699
9799
9899
9999
10099
10199
10299
ID = 122042.0
10400
ID = 122153.0
ID = 122158.0
ID = 122159.0
ID = 122160.0
ID = 122173.0
ID = 122180.0
ID = 122187.0
ID = 122188.0
ID = 122190.0
ID = 122195.0
10510
10610
10710
10810
10910
11010
11110
11210
11310
11410
11510
11610
11710
11810
11910
12010
12110
12210
12310
ID = 124242.0
12411
12511
12611
12711
12811
12911
13011
13111
